In [20]:
def rle_encode(image_array, threshold=0):
    rle_list = []
    current_run_start = None
    current_run_value = None
    current_run_length = 0

    if image_array.ndim == 3:
        # Flatten the image to a 1D array for easier indexing
        image_array = image_array.flatten()

    for i in range(len(image_array)):
        # Threshold the value
        value = 1 if image_array[i] > threshold else 0

        if current_run_start is None:
            current_run_start = i
            current_run_value = value
            current_run_length = 1
        else:
            if value == current_run_value:
                current_run_length += 1
            else:
                rle_list.extend([current_run_start, current_run_value, current_run_length])
                current_run_start = i
                current_run_value = value
                current_run_length = 1

    if current_run_start is not None:
        rle_list.extend([current_run_start, current_run_value, current_run_length])

    return ' '.join(map(str, rle_list))


In [2]:
import os
import pandas as pd
import tensorflow as tf
import time
from PIL import Image 
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam

In [3]:
def load_images_to_array(folder_path,image_list):
    target_size=(224, 224)
    normalize=True
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if filename.lower().endswith(('.tif', '.tiff')):
            # Load the image using Pillow
            img = Image.open(file_path)

            # Resize the image to the target size
            img = img.resize(target_size)

            # Convert the resized image to a numpy array
            img_array = np.array(img)

            # Normalize the pixel values
            if normalize:
                img_array = img_array / 65535.0

            # Append the array to the list
            image_list.append(img_array)

    # Convert the list of arrays to a numpy array
    image_array = np.stack(image_list)

    return image_array

In [6]:
test1_scans = []
test2_scans = []

In [7]:
load_images_to_array("C:\\Users\\youss\\OneDrive - aucegypt.edu\\Desktop\\kidney project\\dataset\\blood-vessel-segmentation\\test\\kidney_5\\images",test1_scans)
load_images_to_array("C:\\Users\\youss\\OneDrive - aucegypt.edu\\Desktop\\kidney project\\dataset\\blood-vessel-segmentation\\test\\kidney_6\\images",test2_scans)



array([[[0.33896391, 0.33531701, 0.33975738, ..., 0.32841993,
         0.32639048, 0.32413214],
        [0.33900969, 0.34077974, 0.33253986, ..., 0.32263676,
         0.32536812, 0.32368963],
        [0.33630884, 0.34029145, 0.33447776, ..., 0.33060197,
         0.3247425 , 0.31946288],
        ...,
        [0.3314107 , 0.33322652, 0.33473716, ..., 0.31023117,
         0.31378653, 0.31747921],
        [0.32663462, 0.32494087, 0.32278935, ..., 0.31621271,
         0.32111086, 0.32173648],
        [0.33836881, 0.33896391, 0.33708705, ..., 0.32443732,
         0.31625849, 0.32088197]],

       [[0.33731594, 0.33652247, 0.33789578, ..., 0.32463569,
         0.32649729, 0.31947814],
        [0.3431754 , 0.34535744, 0.33182269, ..., 0.32755016,
         0.32573434, 0.32385748],
        [0.33524071, 0.34273289, 0.33678187, ..., 0.33116655,
         0.32704662, 0.32054627],
        ...,
        [0.33086137, 0.33313497, 0.32962539, ..., 0.31041428,
         0.31766232, 0.3164416 ],
        [0.3

In [8]:
test1=np.array(test1_scans)

In [9]:
test2=np.array(test2_scans)

In [12]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model("C:\\Users\\youss\\OneDrive - aucegypt.edu\\Desktop\\kidney project\\saved_model_kidney")

# Now you can use the 'model' object for predictions


In [22]:
# Make predictions and encode in RLE format for each image in test1
rle_outputs = []

for img in test2:
    # Add an extra dimension to match the expected input shape of the model
    img = np.expand_dims(img, axis=0)

    # Make prediction
    prediction = model.predict(img)
    
    # Assuming your model outputs a single value per image
    prediction_value = prediction[0]

    # Encode the prediction in RLE format
    rle_output = rle_encode(prediction_value)

    rle_outputs.append(rle_output)

# Print the RLE-encoded outputs
for i, rle_output in enumerate(rle_outputs):
    print(f"Image {i+1} RLE Output: {rle_output}")

1/1 [==============================] - 0s 237ms/step
Image 1 RLE Output: 0 1 3 3 0 218 221 1 4 225 0 222 447 1 2 449 0 222 671 1 1 672 0 48832 49504 1 1 49505 0 222 49727 1 2 49729 0 222 49951 1 4 49955 0 218 50173 1 3
Image 2 RLE Output: 0 1 3 3 0 218 221 1 4 225 0 222 447 1 2 449 0 222 671 1 1 672 0 48832 49504 1 1 49505 0 222 49727 1 2 49729 0 222 49951 1 4 49955 0 218 50173 1 3
Image 3 RLE Output: 0 1 3 3 0 218 221 1 4 225 0 222 447 1 2 449 0 222 671 1 1 672 0 48832 49504 1 1 49505 0 222 49727 1 2 49729 0 222 49951 1 4 49955 0 218 50173 1 3


In [19]:
rle_outputs[0]

'0 0 50176'